In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time
import statsmodels.api as sm
from sklearn import linear_model
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error
from statsmodels.regression.quantile_regression import QuantReg

from sklearn.preprocessing import StandardScaler

import os

from sklearn.model_selection import train_test_split

In [27]:
path = 'data'+os.path.sep
df = pd.read_csv(path+'market_info_soul.csv',encoding='utf-8',delimiter='|')

In [7]:
len(df)

367534

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367534 entries, 0 to 367533
Data columns (total 39 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   상가업소번호     367534 non-null  int64  
 1   상호명        367534 non-null  object 
 2   지점명        56770 non-null   object 
 3   상권업종대분류코드  367534 non-null  object 
 4   상권업종대분류명   367534 non-null  object 
 5   상권업종중분류코드  367534 non-null  object 
 6   상권업종중분류명   367534 non-null  object 
 7   상권업종소분류코드  367534 non-null  object 
 8   상권업종소분류명   367534 non-null  object 
 9   표준산업분류코드   344363 non-null  object 
 10  표준산업분류명    344363 non-null  object 
 11  시도코드       367534 non-null  int64  
 12  시도명        367534 non-null  object 
 13  시군구코드      367534 non-null  int64  
 14  시군구명       367534 non-null  object 
 15  행정동코드      367534 non-null  int64  
 16  행정동명       367458 non-null  object 
 17  법정동코드      367534 non-null  int64  
 18  법정동명       367534 non-null  object 
 19  지번코드       367534 non-n

In [20]:
pd.set_option('display.max_rows',500)

In [10]:
pd.set_option('display.max_columns',100)

In [29]:
df = df[['상호명','상가업소번호','지점명','상권업종대분류명','상권업종중분류명','상권업종소분류명','표준산업분류명','시도명','행정동명','경도','위도']]

In [30]:
df['상권업종대분류명'].unique()

array(['음식', '생활서비스', '소매', '관광/여가/오락', '학문/교육', '부동산', '숙박', '스포츠'],
      dtype=object)

In [31]:
df[df['상권업종대분류명']=='학문/교육']['상권업종중분류명'].unique()

array(['학원-어학', '학원-음악미술무용', '학원-보습교습입시', '학원-예능취미체육', '유아교육', '학원기타',
       '학문교육기타', '학원-자격/국가고시', '학원-컴퓨터', '도서관/독서실', '학원-창업취업취미'],
      dtype=object)

In [52]:
df[df['상권업종대분류명']=='학문/교육'].groupby(['상권업종중분류명','상권업종소분류명']).count()

상호명  상가업소번호  지점명  상권업종대분류명  표준산업분류명   시도명  행정동명  \
상권업종중분류명   상권업종소분류명                                                        
도서관/독서실    고시원           853     853   47       853      853   853   853   
           독서실           914     914   56       914      914   914   914   
           이동도서관운영        18      18    1        18       18    18    18   
유아교육       놀이방           116     116    1       116      116   116   116   
           동화구연            2       2    0         2        2     2     2   
           블럭놀이방          17      17    1        17       17    17    17   
           실내놀이터-유아       71      71   11        71       71    71    71   
           어린이집         3700    3700    7      3700     3700  3700  3700   
           유아교육기관기타       90      90    7        90       90    90    90   
학문교육기타     시험정보안내          5       5    0         5        5     5     5   
           운동/코치학교        53      53    4        53       53    53    53   
           학습지보급         424     424   77       424      424   424   424   
학원-보습교습입시  학원-검정고시         7       7    3         7        7     7     7   
           학원-입시        6706    6706  656      6706     6706  6706  6706   
           학원-편입          24      24   10        24       24    24    24   
학원-어학      학원-어린이영어      306     306   81       306      306   306   306   
           학원-외국어/어학    3269    3269  430      3269     3269  3269  3268   
학원-예능취미체육  검도도장           97      97    1        97       97    97    97   
           레크레이션지도/상담      3       3    0         3        3     3     3   
           유도장            46      46    2        46       46    46    46   
           태권도장          640     640    5       640      640   640   640   
           학원-가요          35      35    3        35       35    35    35   
           학원-기억법          4       4    0         4        4     4     4   
           학원-꽃꽃이         25      25    5        25       25    25    25   
           학원-레크리에이션       1       1    0         1        1     1     1   
           학원-마술           6       6    0         6        6     6     6   
           학원-바둑          34      34    2        34       34    34    34   
           학원-배우/탤런트      84      84   18        84       84    84    84   
           학원-사교춤         46      46   11        46       46    46    46   
           학원-사진           2       2    1         2        2     2     2   
           학원-서당          18      18    2        18       18    18    18   
           학원-스포츠/재즈댄스   244     244   23       244      244   244   244   
           학원-아트플라워        1       1    0         1        1     1     1   
           학원-에어로빅        23      23    2        23       23    23    23   
           학원-요리          67      67   18        67       67    67    67   
           학원-웅변          30      30    7        30       30    30    30   
           학원-자수/동양자수      3       3    0         3        3     3     3   
           학원-주산암산부기속셈   113     113   25       113      113   113   113   
           학원-지점토          1       1    0         1        1     1     1   
           학원-펜글씨/한문      24      24    1        24       24    24    24   
           합기도장          104     104    1       104      104   104   104   
학원-음악미술무용  서예/서화/미술     1473    1473  228      1473     1473  1473  1473   
           피아노/바이올린/기타  2756    2756  341      2756     2756  2756  2755   
           학원-국악          43      43    2        43       43    43    43   
           학원-무용         214     214   37       214      214   214   214   
학원-자격/국가고시 학원-간호조무사       71      71   11        71       71    71    71   
           학원-건축토목기술       7       7    2         7        7     7     7   
           학원-경찰           4       4    3         4        4     4     4   
           학원-공인중개사       17      17    6        17       17    17    17   
           학원-관광통역         2       2    0         2        2     2     2   
           학원-국가고시         7       7    1         7        7     7     7 

### 대 : '학문/교육'

In [53]:
df = df[df['상권업종대분류명']=='학문/교육']

### 중 : 학문교육기타 ,학원-보습교습입시 ,학원-어학

In [63]:
df

,상호명,상가업소번호,지점명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,시도명,행정동명,경도,위도
66,안재형영어,20736806,NaN,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,목5동,126.875973,37.531592
80,정아트,20769711,NaN,학문/교육,학원-음악미술무용,서예/서화/미술,예술 학원,서울특별시,염창동,126.873645,37.550924
82,삼익피아노강동대리점,20927403,강동대리점,학문/교육,학원-음악미술무용,피아노/바이올린/기타,예술 학원,서울특별시,천호3동,127.129075,37.539262
83,설악피아노학원,20966240,NaN,학문/교육,학원-음악미술무용,피아노/바이올린/기타,예술 학원,서울특별시,잠원동,127.015432,37.517844
89,이엠학원,20861438,NaN,학문/교육,학원-보습교습입시,학원-입시,일반 교과 학원,서울특별시,군자동,127.074812,37.554303
...,...,...,...,...,...,...,...,...,...,...,...
367464,클랑미술교습소,16719507,NaN,학문/교육,학원-음악미술무용,서예/서화/미술,예술 학원,서울특별시,고덕2동,127.162422,37.560516
367465,아트앤하트삼익그린미술교습소,16719512,NaN,학문/교육,학원-음악미술무용,서예/서화/미술,예술 학원,서울특별시,명일1동,127.146260,37.551462
367466,학원,16719314,윤선생IGSE영재,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,석관동,127.068979,37.613753
367467,황명주중국어랩,16719291,NaN,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,청림동,126.960977,37.491589


In [64]:
df = df[df['상권업종중분류명'].apply(lambda x : x in ['학문교육기타','학원-보습교습입시','학원-어학']) ]

### 소 : '학원-외국어/어학','학원-입시'

In [67]:
df.groupby(['상권업종소분류명']).count()

,상호명,상가업소번호,지점명,상권업종대분류명,상권업종중분류명,표준산업분류명,시도명,행정동명,경도,위도
상권업종소분류명,,,,,,,,,,
시험정보안내,5,5,0,5,5,5,5,5,5,5
운동/코치학교,53,53,4,53,53,53,53,53,53,53
학습지보급,424,424,77,424,424,424,424,424,424,424
학원-검정고시,7,7,3,7,7,7,7,7,7,7
학원-어린이영어,306,306,81,306,306,306,306,306,306,306
학원-외국어/어학,3269,3269,430,3269,3269,3269,3269,3268,3269,3269
학원-입시,6706,6706,656,6706,6706,6706,6706,6706,6706,6706
학원-편입,24,24,10,24,24,24,24,24,24,24


In [70]:
df = df[df['상권업종소분류명'].apply(lambda x : x in ['학원-외국어/어학','학원-입시'] ) ]

In [71]:
df

,상호명,상가업소번호,지점명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,시도명,행정동명,경도,위도
66,안재형영어,20736806,NaN,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,목5동,126.875973,37.531592
89,이엠학원,20861438,NaN,학문/교육,학원-보습교습입시,학원-입시,일반 교과 학원,서울특별시,군자동,127.074812,37.554303
115,삼성수학,21055207,NaN,학문/교육,학원-보습교습입시,학원-입시,일반 교과 학원,서울특별시,대치2동,127.062607,37.500441
135,지엔비화곡외국어학원,21867091,NaN,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,화곡본동,126.845150,37.545693
150,김찬영어전문학원,21883171,NaN,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,화곡6동,126.848757,37.550306
...,...,...,...,...,...,...,...,...,...,...,...
367441,논술학원,16720483,NaN,학문/교육,학원-보습교습입시,학원-입시,일반 교과 학원,서울특별시,창4동,127.045967,37.660450
367459,학원,16719328,비오엘잉글리시원격,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,신촌동,126.929609,37.557986
367460,해법중국어원효점교습소,16719322,NaN,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,원효로2동,126.950049,37.533851
367466,학원,16719314,윤선생IGSE영재,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,석관동,127.068979,37.613753


In [48]:
df[df['상호명'].str.contains('영어')]

,상호명,상가업소번호,지점명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,시도명,행정동명,경도,위도
66,안재형영어,20736806,NaN,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,목5동,126.875973,37.531592
150,김찬영어전문학원,21883171,NaN,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,화곡6동,126.848757,37.550306
311,오케이영어,22745352,NaN,학문/교육,학원-보습교습입시,학원-입시,일반 교과 학원,서울특별시,명일2동,127.155115,37.549030
346,튼튼영어용산이촌지사,22324203,NaN,학문/교육,학문교육기타,학습지보급,방문 교육 학원,서울특별시,이촌1동,126.977856,37.518443
475,다니엘해법영어교실,23327603,NaN,학문/교육,학원-어학,학원-어린이영어,외국어학원,서울특별시,자양3동,127.071496,37.535156
...,...,...,...,...,...,...,...,...,...,...,...
363503,클라이영어교습소,16760687,NaN,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,장위3동,127.050668,37.619653
363504,코틀린영어교습소,16760450,NaN,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,목5동,126.877967,37.535954
363510,케이트영어,16760428,NaN,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,일원2동,127.075177,37.493656
363747,그루영어,16731088,NaN,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,반포4동,127.010507,37.502764


In [39]:
df[ df['상권업종대분류명'] =='학문/교육' | df['상호명'].str.contains('플래티넘') ]

TypeError: Cannot perform 'ror_' with a dtyped [bool] array and scalar of type [bool]